In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings('ignore')

In this lab, you will be using the Sakila database of movie rentals.

In order to optimize our inventory, we would like to know which films will be rented next month and we are asked to create a model to predict it.

#### Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features.

#### Read the data into a Pandas dataframe.

In [2]:
from mysql.connector import connect
from getpass import getpass

In [3]:
db = connect(
        host= "localhost",
        user = "root",
        passwd = getpass("Enter password: "),
        database= "sakila"
   )

Enter password:  ········


In [4]:
cursor = db.cursor()

In [5]:
rentals_sql = "select * from python_table;"

In [6]:
predictions = pd.read_sql_query(rentals_sql,db)

In [7]:
predictions.head(5)

,title,film_id,Frequency,release_year,length,Rental_Date,Return_Date,rental_duration,category
0,ZORRO ARK,1000,2005-07-12,2006,50,2005-07-12,2005-07-16,3,Comedy
1,ZORRO ARK,1000,2005-07-29,2006,50,2005-07-29,2005-08-06,3,Comedy
2,ZORRO ARK,1000,2005-08-19,2006,50,2005-08-19,2005-08-27,3,Comedy
3,ZORRO ARK,1000,2005-06-17,2006,50,2005-06-17,2005-06-24,3,Comedy
4,ZORRO ARK,1000,2005-07-07,2006,50,2005-07-07,2005-07-08,3,Comedy


#### Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables.


In [8]:
predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16045 entries, 0 to 16044
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            16045 non-null  object
 1   film_id          16045 non-null  int64 
 2   Frequency        16045 non-null  object
 3   release_year     16045 non-null  int64 
 4   length           16045 non-null  int64 
 5   Rental_Date      16045 non-null  object
 6   Return_Date      15862 non-null  object
 7   rental_duration  16045 non-null  int64 
 8   category         16045 non-null  object
dtypes: int64(4), object(5)
memory usage: 1.1+ MB


In [9]:
predictions.shape

(16045, 9)

In [10]:
predictions.Rental_Date.sort_values(ascending=False)

16016    2022-08-24
4824     2006-02-14
11286    2006-02-14
13633    2006-02-14
2052     2006-02-14
            ...    
7843     2005-05-24
8774     2005-05-24
2075     2005-05-24
14769    2005-05-24
6248     2005-05-24
Name: Rental_Date, Length: 16045, dtype: object

In [11]:
predictions.isna().sum()

title                0
film_id              0
Frequency            0
release_year         0
length               0
Rental_Date          0
Return_Date        183
rental_duration      0
category             0
dtype: int64

#### Create a query to get the list of films and a boolean indicating if it was rented last month. This would be our target variable.


In [12]:
#Convert into to datetime to do boolean
import datetime as dt 

predictions[['Rental_Date','Return_Date']] = predictions[['Rental_Date','Return_Date']].apply(pd.to_datetime)


In [13]:
predictions["Rented_last_month"] = (predictions['Rental_Date'] >= '2006-02-01') & (predictions['Rental_Date'] < '2006-03-01')

In [14]:
predictions.Rented_last_month.value_counts()

False    15863
True       182
Name: Rented_last_month, dtype: int64

In [15]:
predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16045 entries, 0 to 16044
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   title              16045 non-null  object        
 1   film_id            16045 non-null  int64         
 2   Frequency          16045 non-null  object        
 3   release_year       16045 non-null  int64         
 4   length             16045 non-null  int64         
 5   Rental_Date        16045 non-null  datetime64[ns]
 6   Return_Date        15862 non-null  datetime64[ns]
 7   rental_duration    16045 non-null  int64         
 8   category           16045 non-null  object        
 9   Rented_last_month  16045 non-null  bool          
dtypes: bool(1), datetime64[ns](2), int64(4), object(3)
memory usage: 1.1+ MB


In [16]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
predictions.title = le.fit_transform(predictions.title)
predictions.Frequency = le.fit_transform(predictions.Frequency)
predictions.category = le.fit_transform(predictions.category)

In [17]:
predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16045 entries, 0 to 16044
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   title              16045 non-null  int64         
 1   film_id            16045 non-null  int64         
 2   Frequency          16045 non-null  int64         
 3   release_year       16045 non-null  int64         
 4   length             16045 non-null  int64         
 5   Rental_Date        16045 non-null  datetime64[ns]
 6   Return_Date        15862 non-null  datetime64[ns]
 7   rental_duration    16045 non-null  int64         
 8   category           16045 non-null  int64         
 9   Rented_last_month  16045 non-null  bool          
dtypes: bool(1), datetime64[ns](2), int64(7)
memory usage: 1.1 MB


#### Create a logistic regression model to predict this variable from the cleaned data.

In [23]:
X=predictions.drop(["Rented_last_month","Rental_Date","Return_Date"], axis=1)
y=predictions["Rented_last_month"]

In [24]:
#train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [25]:
# train a logistic model
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [26]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression


model = LogisticRegression()
model.fit(X_train,y_train)

pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

#### Evaluate the results.


In [28]:
print("Train Score : {}".format(model.score(X_train, y_train)))
print('Test Score : {}'.format(model.score(X_test, y_test)))

Train Score : 0.9884699283265815
Test Score : 0.9890931754440636


In [29]:
print(classification_report(y_test,pred_test))
print(classification_report(y_train, pred_train))

              precision    recall  f1-score   support

       False       0.99      1.00      0.99      3174
        True       0.00      0.00      0.00        35

    accuracy                           0.99      3209
   macro avg       0.49      0.50      0.50      3209
weighted avg       0.98      0.99      0.98      3209

              precision    recall  f1-score   support

       False       0.99      1.00      0.99     12689
        True       0.00      0.00      0.00       147

    accuracy                           0.99     12836
   macro avg       0.49      0.50      0.50     12836
weighted avg       0.98      0.99      0.98     12836

